In [54]:
!pip install pandas
!pip install geopandas
!pip install lxml
!pip install urllib


   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   - -------------------------------------- 0.1/3.8 MB 3.8 MB/s eta 0:00:01
   ---- ----------------------------------- 0.4/3.8 MB 5.2 MB/s eta 0:00:01
   ------ --------------------------------- 0.6/3.8 MB 5.8 MB/s eta 0:00:01
   ----------- ---------------------------- 1.1/3.8 MB 6.1 MB/s eta 0:00:01
   --------------- ------------------------ 1.4/3.8 MB 6.5 MB/s eta 0:00:01
   ------------------ --------------------- 1.8/3.8 MB 6.7 MB/s eta 0:00:01
   --------------------- ------------------ 2.0/3.8 MB 6.9 MB/s eta 0:00:01
   ------------------------- -------------- 2.4/3.8 MB 7.0 MB/s eta 0:00:01
   ----------------------------- ---------- 2.8/3.8 MB 7.1 MB/s eta 0:00:01
   -------------------------------- ------- 3.1/3.8 MB 7.0 MB/s eta 0:00:01
   ------------------------------------ --- 3.4/3.8 MB 7.1 MB/s eta 0:00:01
   ---------------------------------------  3.8/3.8 MB 7.1 MB/s eta 0:00:01
   ----------------

ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib


In [1]:
import importlib
import pandas as pd
import requests
import geopandas as gpd
from os.path import isfile
import declarations as dec

### Téléchargement des données de consommation d'électricité annuelle par adresse

In [2]:
def get_data_consumption(url, year, replace:bool = False):
    '''Gets the data of home's electricity consumption
    Args:
        url(string)
        year(string)
        replace(bool): True to replace consommation{year}.csv if it exists. Defaults to False.
    Returns:
        df (dataframe): dataframe of the home's electricity consumption for the wanted year 
    '''
    path_to_data="consommation"+f"{year}"+".csv"
    if (isfile(path_to_data) and not replace):
        df=pd.read_csv(path_to_data, sep=";")
    else:
        print("Chargement des données, cette étape peut prendre quelques minutes")
        response=requests.get(url)
        if response.status_code == 200:
            with open(path_to_data, "wb") as file:
                file.write(response.content)
            print("Téléchargement réussi.")
        else:
            print(f"Échec du téléchargement. Code d'état : {response.status_code}")
        df=pd.read_csv(path_to_data, sep=";")
    return df


In [1]:
# URL des bases de données à charger 

consumption_data_url_2018="https://enedis.opendatasoft.com/api/explore/v2.1/catalog/datasets/consommation-annuelle-residentielle-par-adresse/exports/csv?lang=fr&refine=annee%3A%222018%22&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
consumption_data_url_2019= "https://enedis.opendatasoft.com/api/explore/v2.1/catalog/datasets/consommation-annuelle-residentielle-par-adresse/exports/csv?lang=fr&refine=annee%3A%222019%22&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
consumption_data_url_2020="https://enedis.opendatasoft.com/api/explore/v2.1/catalog/datasets/consommation-annuelle-residentielle-par-adresse/exports/csv?lang=fr&refine=annee%3A%222020%22&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
consumption_data_url_2021="https://enedis.opendatasoft.com/api/explore/v2.1/catalog/datasets/consommation-annuelle-residentielle-par-adresse/exports/csv?lang=fr&refine=annee%3A%222021%22&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"

In [3]:
df_cons_2018=get_data_consumption(consumption_data_url_2018, "2018")
df_cons_2019=get_data_consumption(consumption_data_url_2019, "2019")
df_cons_2020=get_data_consumption(consumption_data_url_2020, "2020")
df_cons_2021=get_data_consumption(consumption_data_url_2021, "2021")

NameError: name 'consumption_data_url_2018' is not defined

### Récupération des données de Diagnostics de Performance Energétique (DPE)

In [4]:
def get_dpe_from_url(api_url):
    '''Imports a database from an API url.
    Args:
        api_url (string) : url of the wanted API
    Returns:
        pandas.DataFrame : dataframe
    '''
    req=requests.get(api_url)
    wb=req.json()
    df = pd.json_normalize(wb["results"])
    dpe = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs = 4326)
    dpe = dpe.dropna(subset = ['longitude', 'latitude'])
    return dpe

In [21]:
# Il est trop couteux de récupérer l'intégralité des données et nous ne disposons de données météo que pour
# certaines communes. L'idée est donc de ne récupérer les données de DPE que pour les communes voulues et de
# les agréger dans une unique table de données.

def get_dpe_dataset(api_root, communes):
    '''
    Args : 
    Returns : 
    '''


### Récupération des données de consommation annuelle d'électricité et gaz par département et par secteur d'activité

In [5]:
def get_data_consumption_department(df, year):
    '''Gets the data of the annual electricity consumption per "département" for a specific year
    Args:
        df (DataFrame)
        year(string)
    Returns:
        df (dataframe): dataframe of the home's electricity consumption for the wanted year 
    '''
    df_year= df[(df["Année"]==year) & (df["Filière"]=="Electricité")]
    return df_year


In [7]:
# Base de données 
cons_département="https://opendata.agenceore.fr/api/explore/v2.1/catalog/datasets/conso-elec-gaz-annuelle-par-secteur-dactivite-agregee-departement/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
df_cons_département=pd.read_csv(cons_département, sep=";")


In [8]:
variables_cons_département= ['Opérateur', 'Année', 'Filière', 
                             'Consommation Résidentiel  (MWh)', 'Nombre de points Résidentiel', 
                             'Nombre de mailles secretisées (résidentiel)','Indice qualité Résidentiel',
                             'Code Département', 'Libellé Département','Code Région', 'Libellé Région', 
                             'id_filiere', 'geom','Consommation totale (MWh)']

In [9]:
list_années=pd.unique(df_cons_département["Année"])
dico_cons_département={} # Création d'un dictionnaire qui à chaque année associe le dataframe de consommation d'électricité résidentielle par départements
for année in list_années:
    if année not in dico_cons_département:
        df_année=get_data_consumption_department(df_cons_département, année)
        dico_cons_département[année]=dec.df_filter(df_année, variables_cons_département)


In [28]:
dico_cons_département[2011]

,Opérateur,Année,Filière,Consommation Résidentiel (MWh),Nombre de points Résidentiel,Nombre de mailles secretisées (résidentiel),Indice qualité Résidentiel,Code Département,Libellé Département,Code Région,Libellé Région,id_filiere,geom,Consommation totale (MWh)
0,Enedis,2011,Electricité,1472026.600,259239,2,0.301,02,Aisne,32,Hauts-de-France,100,NaN,2855280.660
1,Enedis,2011,Electricité,1130822.230,219826,0,0.285,03,Allier,84,Auvergne-Rhône-Alpes,100,NaN,2017190.640
2,Enedis,2011,Electricité,685956.960,131064,0,0.322,04,Alpes-de-Haute-Provence,93,Provence-Alpes-Côte d'Azur,100,NaN,1055358.030
3,Enedis,2011,Electricité,561238.840,123947,0,0.311,05,Hautes-Alpes,93,Provence-Alpes-Côte d'Azur,100,NaN,937412.370
4,Enedis,2011,Electricité,962384.810,169748,1,0.298,10,Aube,44,Grand Est,100,NaN,1897584.710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3791,Régie municipale d'électricité de Varilhes,2011,Electricité,9080.000,1721,0,1.000,09,Ariège,76,Occitanie,100,NaN,14094.000
3792,Régie municipale de Presle,2011,Electricité,1331.000,277,0,0.000,73,Savoie,84,Auvergne-Rhône-Alpes,100,NaN,1372.000
3793,Régie municipale gaz-électricité de Carmaux,2011,Electricité,27681.000,6198,0,0.500,81,Tarn,76,Occitanie,100,NaN,34009.000
3794,Régie municipale multiservices de La Réole,2011,Electricité,4735.830,1147,0,0.800,33,Gironde,75,Nouvelle-Aquitaine,100,NaN,8524.270


In [27]:
# On somme les valeurs de consommations des différents opérateurs pour avoir une valeur unique de consommation
# résidentielle d'électricité par départements.

df_test= dico_cons_département[2011].groupby('Code Département')['Consommation Résidentiel  (MWh)'].sum().reset_index()
df_test.head()

,Code Département,Consommation Résidentiel (MWh)
0,01,1977574.54
1,02,1472026.60
2,03,1130822.23
3,04,685956.96
4,05,561238.84
